# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flows for the REMIND electricity sector, both on a per-year and aggregated level.

In [1]:
%run initialize_notebook.ipynb

In [2]:
%run REMIND_Functions_to_modify_ecoinvent.ipynb

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [3]:
import_karma()

Database Carma CCS already present.


In [4]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

In [19]:
def lca_for_multiple_techs_and_regions(techs, regions, db):
    """ Perform LCA calculations for multiple technologies (activities) and regions.
        The demand is distributed evenly over all found activities (average).
    """
    if len(techs) == 0:
        return None
    print("LCA for activities (from {}): {}".format(db.name, techs))
    actvts = [act for act in db if act["name"] in techs and
                act["location"] in regions]
    if len(actvts) == 0:
        actvts = [act for act in carmadb if act["name"] in techs and
                    act["location"] == "RoW"]
        if len(actvts) == 0:
            actvts = [act for act in carmadb if act["name"] in techs and
                        act["location"] == "GLO"]
            if len(actvts) == 0:
                print("Could not find any activities matching {}".format(techs))
                return None
    
    lca = LCA({act: 1./len(actvts) for act in actvts})
    lca.lci()
    
    return lca



In [24]:
def rmnd_bioflows(region, tech_primers={}):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
        A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
        REMIND technology x.
    """
    carma_techs = ['Biomass IGCC CCS', 'Biomass IGCC', 'Coal IGCC', 'Coal IGCC CCS', 'Coal PC CCS', 'Gas CCS']
    carmadb = Database("Carma CCS")
    eidb = Database("ecoinvent_3.5")
    result = {}
    bio_names = []
    regions = [el[1] if type(el) == tuple else el for el in geomatcher.contained(("REMIND", region))]
    if region == "EUR":
        regions.append("RER")
    print("Found following regions for {}: {}".format(region, regions))
    for key in available_electricity_generating_technologies:
        if key in tech_primers:
            techs = [tech_primers[key]]
        else:
            techs = available_electricity_generating_technologies[key]
        if len(techs) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
        if key in carma_techs:
            lca = lca_for_multiple_techs_and_regions(techs, regions, carmadb)
        else:
            lca = lca_for_multiple_techs_and_regions(techs, regions, eidb)

        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        result[key] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [21]:
df = rmnd_bioflows("EUR")

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
LCA for activities (from Carma CCS): ['Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025', 'Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025', 'Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025']
Bionames changed, old length: 0
new length: 2079
LCA for activities (from Carma CCS): ['Electricity, at BIGCC power plant 450MW, no CCS/2025']
LCA for activities (from Carma CCS): ['Electricity, at power plant/hard coal, IGCC, no CCS/2025', 'Electricity, at power plant/lignite, IGCC, no CCS/2025']
LCA for activities (from Carma CCS): ['Electricity, at power plant/hard coal, pre, pipeline 200km, sto

In [22]:
df.sort_index()[:30]

amount
            flow                                               
Biomass CHP 1,3-Dioxolan-2-one                     5.695489e-10
            1,4-Butanediol                         6.603185e-12
            1-Pentanol                             6.221941e-13
            1-Pentene                              8.222725e-12
            2,2,4-Trimethyl pentane                4.485510e-15
            2,4-D                                  6.836292e-09
            2,4-D amines                           7.790224e-20
            2,4-D ester                            5.174172e-20
            2,4-DB                                 5.166284e-20
            2,4-di-tert-butylphenol                0.000000e+00
            2-Aminopropanol                        5.971345e-13
            2-Methyl pentane                       7.202318e-10
            2-Methyl-1-propanol                    1.837457e-12
            2-Methyl-2-butene                      3.326384e-16
            2-Nitrobenzoic acid                    3.131498e-13
            2-Propanol                             1.729355e-08
            2-chlorobenzaldehyde                   0.000000e+00
            3-Methyl-1-butanol                     0.000000e+00
            4-Methyl-2-pentanol                    1.284421e-19
            4-Methyl-2-pentanone                   1.805261e-12
            AOX, Adsorbable Organic Halogen as Cl  4.156014e-09
            Abamectin                              0.000000e+00
            Acenaphthene                           1.485787e-12
            Acenaphthylene                         1.287988e-13
            Acephate                               1.625737e-10
            Acetaldehyde                           3.709902e-07
            Acetamide                              2.389007e-11
            Acetamiprid                            0.000000e+00
            Acetic acid                            5.246549e-08
            Acetochlor                             1.202184e-13